This Python notebook file will generate a thin film of ferroelectric material in MOOSE at a given temperature. Here we will exploit the mesh construction in CUBIT and then pass the information about the material constants to an input file suitable for MOOSE/Ferret.

In [2]:
XY_size = '20'

film_height = '12'

substrate_depth = '10'

move_vector = int(film_height)/2.0 + int(substrate_depth)/2.0

# input mesh size less than 1.0
mesh_size = 0.9

In [3]:
import os
import subprocess
import numpy as np

os.chdir('/home/john/projects/ferret/python')

def replace_all(text, dic):
    for i, j in dic.iteritems():
        text = text.replace(i, j)
    return text

line_one = 'create brick x XY y XY z FilmHeight'
line_two = 'create brick x XY y XY z SubstrateDepth'
line_three = 'volume 2 move 0 0 MoveVector'
line_four = 'volume 1 2 size MeshSize'
line_five = 'export Genesis "./exodus_thinfilm_test_def.e" dimension 3 block all overwrite'

replacements_line_one = {'XY': XY_size, 'FilmHeight': film_height}
replacements_line_two = {'XY': XY_size, 'SubstrateDepth': substrate_depth}
replacements_line_three = {'MoveVector': repr(move_vector)}
replacements_line_four = {'MeshSize': repr(mesh_size)}
replacements_line_five = {"./exodus_thinfilm_test_def.e": "/home/john/projects/ferret/mesh/./exodus_thinfilm_0" 
                          + repr(int(round(10.0 * mesh_size))) 
                          + "_" + repr(int(XY_size))
                          + "_" + repr(int(film_height))
                          + "_" + repr(int(substrate_depth))
                          + ".e"
                         }

replace_line_one = replace_all(line_one, replacements_line_one)
replace_line_two = replace_all(line_two, replacements_line_two)
replace_line_three = replace_all(line_three, replacements_line_three)
replace_line_four = replace_all(line_four, replacements_line_four)
replace_line_five = replace_all(line_five, replacements_line_five)

# Define the base file we will be modifiying #
infile = open('/home/john/projects/ferret/meshrecipes/def/thinfilmDef')
outfile = open('/home/john/projects/ferret/meshrecipes/thinfilm.py', 'w')

replacements = {line_one:replace_line_one,
                line_two:replace_line_two,
                line_three:replace_line_three,
                line_four:replace_line_four,
                line_five:replace_line_five
               }
for line in infile:
    for src, target in replacements.iteritems():
        line = line.replace(src, target)
    outfile.write(line)
infile.close()
outfile.close()

Now we take the above file and pass it to the CUBIT and save the resulting mesh file in /ferret/mesh

In [ ]:
import os
import subprocess

os.chdir('/home/john/projects/cubit/cubit-14.1/bin')
subprocess.call('./cubit /home/john/projects/ferret/meshrecipes/thinfilm.py',shell=True)